In [1]:
import datetime
import xgboost
import numpy as np
import pandas as pd
import utils.data_loader as data_loader
import utils.display as display
import utils.vector_haversine_distances as vec_hs_dis
from scipy import stats
from sklearn.metrics.pairwise import haversine_distances
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sklearn
from hotspot.hotpots_discovery_utils import generate_cube_index, cube_to_coordinate
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import make_scorer

In [2]:
display.configure_logging()
display.configure_pandas()

In [3]:
def mape_vectorized_v2(a, b):
    b = b.reshape(1, -1)
    mask = a != 0
    a = a[mask]
    b = b[mask]
    return (np.fabs(a - b)/a).mean()

In [4]:
df_od = data_loader.load_od(scale='full', common=False)
# 筛选出处于bbox中的points
df_od['in_bbox'] = ((113.764635 < df_od['destination_log'])
                     & (df_od['destination_log'] < 114.608972)
                     & (22.454727 < df_od['destination_lat'])
                     & (df_od['destination_lat'] < 22.842654)
                     & (113.764635 < df_od['original_log'])
                     & (df_od['original_log'] < 114.608972)
                     & (22.454727 < df_od['original_lat'])
                     & (df_od['original_lat'] < 22.842654))
df_od = df_od.loc[df_od.in_bbox].reset_index(drop=True)
df_od = generate_cube_index(df_od, m=100, n=200)

demand = df_od.groupby(['original_cube', 'destination_cube']).size().reset_index()
demand = demand.rename(columns={0: 'demand'})

demand = demand.loc[demand['demand'] > 10].reset_index()

24-Jul-20 15:33:18 - Loading data/transaction_201407.csv


In [5]:
df_et_od = data_loader.load_od(scale='full', common=True)
# 筛选出处于bbox中的points
df_et_od['in_bbox'] = ((113.764635 < df_et_od['destination_log'])
                     & (df_et_od['destination_log'] < 114.608972)
                     & (22.454727 < df_et_od['destination_lat'])
                     & (df_et_od['destination_lat'] < 22.842654)
                     & (113.764635 < df_et_od['original_log'])
                     & (df_et_od['original_log'] < 114.608972)
                     & (22.454727 < df_et_od['original_lat'])
                     & (df_et_od['original_lat'] < 22.842654))
df_et_od = df_et_od.loc[df_et_od.in_bbox].reset_index(drop=True)
df_et_od = generate_cube_index(df_et_od, m=100, n=200)

et_demand = df_et_od.groupby(['original_cube', 'destination_cube']).size().reset_index()
et_demand = et_demand.rename(columns={0: 'demand'})

24-Jul-20 15:33:48 - Loading data/transaction_common_201407.csv


In [6]:
df_demands = pd.merge(demand, et_demand, how='left', on=['original_cube', 'destination_cube'], suffixes=('_all', '_et'))
df_demands = df_demands.fillna(0)
df_demands['rate'] = df_demands['demand_et'] / df_demands['demand_all']

In [7]:
df_od['duration'] = (df_od['end_time'] - df_od['begin_time']).dt.total_seconds()
df_od = df_od[['original_cube', 'destination_cube', 'original_log', 'original_lat', 'destination_log',
               'destination_lat', 'duration']].groupby(['original_cube', 'destination_cube']).mean()
df_od.reset_index(inplace=True)

In [8]:
df_od_pairs = pd.merge(df_demands[['original_cube', 'destination_cube']],
                       df_od[['original_cube', 'destination_cube', 'original_log', 'original_lat', 'destination_log',
                              'destination_lat', 'duration']],
                       left_on=['original_cube', 'destination_cube'],
                       right_on=['original_cube', 'destination_cube'])

# df_od_pairs = df_od_pairs.sample(frac=1).reset_index(drop=True)

df_cs, date = data_loader.load_cs(scale='part', date=datetime.datetime(2014, 7, 1))
df_cs = df_cs.loc[~df_cs['cs_name'].isin(['LJDL', 'E04', 'BN0002', 'F11', 'S1', 'S2', 'F12', 'F13', 'F15'])]
df_cs.reset_index(drop=True, inplace=True)

# select drop location and CS location as two array
original_locations = df_od_pairs[['original_lat', 'original_log']].to_numpy()
destination_locations = df_od_pairs[['destination_lat', 'destination_log']].to_numpy()
cs_location = df_cs[['Latitude', 'Longitude']].to_numpy()

# earth radius(km)
AVG_EARTH_RADIUS = 6371.0088

# calculate distance between original/destination location and CS location
original_distances_to_cs = haversine_distances(np.radians(original_locations), np.radians(cs_location)) \
                           * AVG_EARTH_RADIUS
destination_distances_to_cs = haversine_distances(np.radians(destination_locations), np.radians(cs_location)) \
                              * AVG_EARTH_RADIUS
# calculate distance between od locations
od_dis = vec_hs_dis.haversine_np(df_od_pairs['original_log'], df_od_pairs['original_lat'],
                                 df_od_pairs['destination_log'], df_od_pairs['destination_lat'])

df_original_distances_to_cs = pd.DataFrame(original_distances_to_cs)
df_destination_distances_to_cs = pd.DataFrame(destination_distances_to_cs)
df_od_dis = pd.DataFrame(od_dis)

24-Jul-20 15:33:50 - Loadingdata/cs/ChargeLocation201407


In [159]:
def evaluate(n=10):
    original_capacity = np.repeat(df_cs['chg_points'].values.reshape(1, -1), df_od_pairs.shape[0], axis=0)

    a = df_original_distances_to_cs.values
    a.sort(axis=1)
    a = pd.DataFrame(a, df_original_distances_to_cs.index, df_original_distances_to_cs.columns)
    a = a.iloc[:, :n]
    o_dissorted_capacity = np.take_along_axis(original_capacity, df_original_distances_to_cs.values.argsort(axis=1),
                                              axis=1)
    o_dissorted_capacity = pd.DataFrame(o_dissorted_capacity)
    o_dissorted_capacity = o_dissorted_capacity.iloc[:, :n]

    b = df_destination_distances_to_cs.values
    b.sort(axis=1)
    b = pd.DataFrame(b, df_destination_distances_to_cs.index, df_destination_distances_to_cs.columns)
    b = b.iloc[:, :n]
    d_dissorted_capacity = np.take_along_axis(original_capacity, df_destination_distances_to_cs.values.argsort(axis=1),
                                              axis=1)
    d_dissorted_capacity = pd.DataFrame(d_dissorted_capacity)
    d_dissorted_capacity = d_dissorted_capacity.iloc[:, :n]

    # train_x, val_x, train_y, val_y = train_test_split(
    #     pd.concat([df_od_pairs['duration'], od_dis, a, o_dissorted_capacity, b, d_dissorted_capacity],axis=1).values,
    #     df_demands['rate'].values, test_size=0.2)
    data_set = pd.concat([df_od_pairs['duration'], od_dis, a, o_dissorted_capacity, b, d_dissorted_capacity,
               df_demands['demand_et'], df_demands['rate']], axis=1)
    data_set = data_set.loc[data_set['demand_et'] > 5]
    train_x, val_x, train_y, val_y = train_test_split(data_set.iloc[:, :-2].values,
                                                      data_set.iloc[:, -1].values, test_size=0.2)
    print('train data shape:', train_x.shape)
    print('test data shape:', val_x.shape)
    scaler = sklearn.preprocessing.StandardScaler()
    train_x = scaler.fit_transform(train_x)
    val_x = scaler.transform(val_x)

    gbm = xgboost.XGBRegressor(verbosity=0, n_estimators=100, validate_parameters=2, 
                               learning_rate=0.05, min_child_weight=5, max_depth=8)

    gbm.fit(train_x, train_y)
    predict_y = gbm.predict(val_x)

    print('mae:', mean_absolute_error(val_y, predict_y),
          'mse:', mean_squared_error(val_y, predict_y),
          'rmse:', mean_squared_error(val_y, predict_y, squared=False))
    print('mape:', mape_vectorized_v2(val_y.reshape(1, -1), predict_y))
    print('R2:', gbm.score(val_x, val_y))
    print(val_y[30:36], )
    print(predict_y[30:36])
    print(val_y[50:56], )
    print(predict_y[50:56])

print(df_od_pairs.shape[0], 'sample')
for i in range(3, 4):
    print(i, 'nearest cs as feature:')
    evaluate(n=i)

106788 sample
3 nearest cs as feature:
train data shape: (4003, 14)
test data shape: (1001, 14)
mae: 0.026461035292838 mse: 0.0020419696121095955 rmse: 0.04518815787470867
mape: 0.3501005397370062
R2: 0.5141357348305923
[0.12280702 0.04511278 0.03831418 0.03508772 0.05108055 0.14851485]
[0.09802815 0.05021524 0.05168721 0.10367793 0.05502495 0.12419114]
[0.04051173 0.12195122 0.04009434 0.10526316 0.06593407 0.05940594]
[0.05017522 0.06613648 0.05212858 0.10769045 0.08858252 0.07811704]


In [113]:
df_demands.describe()

,index,original_cube,destination_cube,demand_all,demand_et,rate
count,106788.000000,106788.000000,106788.00000,106788.000000,106788.000000,106788.000000
mean,579180.341508,13816.367082,13778.56040,36.811393,1.383835,0.037035
std,264718.064414,2694.106499,2705.44901,52.576251,2.785713,0.049223
min,1708.000000,2830.000000,3229.00000,11.000000,0.000000,0.000000
25%,373768.750000,12483.000000,12466.00000,14.000000,0.000000,0.000000
50%,622343.500000,14684.000000,14666.00000,21.000000,1.000000,0.017857
75%,802127.250000,15656.000000,15651.00000,38.000000,2.000000,0.062500
max,994057.000000,18634.000000,19031.00000,3672.000000,129.000000,0.823529


In [15]:
exit(0)
# This cell is used to search parameters.
def evaluate(n=10):
    original_capacity = np.repeat(df_cs['chg_points'].values.reshape(1, -1), df_od_pairs.shape[0], axis=0)

    a = df_original_distances_to_cs.values
    a.sort(axis=1)
    a = pd.DataFrame(a, df_original_distances_to_cs.index, df_original_distances_to_cs.columns)
    a = a.iloc[:, :n]
    o_dissorted_capacity = np.take_along_axis(original_capacity, df_original_distances_to_cs.values.argsort(axis=1),
                                              axis=1)
    o_dissorted_capacity = pd.DataFrame(o_dissorted_capacity)
    o_dissorted_capacity = o_dissorted_capacity.iloc[:, :n]

    b = df_destination_distances_to_cs.values
    b.sort(axis=1)
    b = pd.DataFrame(b, df_destination_distances_to_cs.index, df_destination_distances_to_cs.columns)
    b = b.iloc[:, :n]
    d_dissorted_capacity = np.take_along_axis(original_capacity, df_destination_distances_to_cs.values.argsort(axis=1),
                                              axis=1)
    d_dissorted_capacity = pd.DataFrame(d_dissorted_capacity)
    d_dissorted_capacity = d_dissorted_capacity.iloc[:, :n]

    gbm = xgboost.XGBRegressor(verbosity=0, validate_parameters=2,)

    param_grid = {
        'n_estimators': [50, 100],
        'scale_pos_weight': [1, 3],
        'max_depth': [5, 10],
        'learning_rate': [0.05, 0.1],
        'min_child_weight': [1, 3],
        'gamma': [0, 5,],
        'max_delta_step': [0, 3],
        'subsample': [0.7, 1],
        'colsample_bytree': [0.7, 1]
    }
    gs = GridSearchCV(gbm, param_grid=param_grid, cv=5)

    train_x, val_x, train_y, val_y = train_test_split(
        pd.concat([df_od_pairs['duration'], od_dis, a, o_dissorted_capacity, b, d_dissorted_capacity],axis=1).values,
        df_demands['rate'].values, test_size=0.2)
    print(train_x.shape, train_y.shape)
    print(val_x.shape, val_y.shape)
    scaler = sklearn.preprocessing.StandardScaler()
    train_x = scaler.fit_transform(train_x)
    val_x = scaler.transform(val_x)

    gs.fit(train_x, train_y)

    print(gs.best_params_)
    print(gs.best_score_)
    print(gs.best_estimator_)
    print(gs.best_index_)

    gs.best_estimator_.fit(train_x, train_y)
    predict_y = gs.best_estimator_.predict(val_x)

    print('mape:', mape_vectorized_v2(val_y.reshape(-1), predict_y))
    print('xgb score (rmse in xgb doc):', gs.best_estimator_.score(val_x, val_y))
    print('gs score (neg rmse):', gs.score(val_x, val_y))
    print('sample_gt:', val_y[10:15])
    print('sample_pred:', predict_y[10:15])

print(df_od_pairs.shape[0], 'sample')
for i in range(1, 10):
    print(i, 'nearest cs as feature:')
    evaluate(n=i)

106788 sample
1 nearest cs as feature:
(85430, 6) (85430,)
(21358, 6) (21358,)


KeyboardInterrupt: 